# Network structure

Can network structure explain the heterogeneity in the results?

In [143]:
using KFactors, Graphs, OpenStreetMapGraphBuilder, CSV, DataFrames, MetaGraphs, LibSpatialIndex,
    LibGEOS, StatsBase, Serialization, ArchGDAL, SpatialDependence, SparseArrays, Pipe

In [78]:
const SEARCH_RADIUS_METERS = 1000
const METERS_PER_DEGREE_LAT = 90 / 10_000_000
const SEARCH_RADIUS_DEGREES_LAT = SEARCH_RADIUS_METERS * METERS_PER_DEGREE_LAT

0.009000000000000001

In [79]:
way_filter(way) = haskey(way.tags, "highway") && (
        # will overinclude trunk and primary link roads on non-State Highway System routes,
        # but that shouldn't hurt anything
        way.tags["highway"] ∈ Set(["motorway", "motorway_link", "trunk_link", "primary_link"]) ||
        (way.tags["highway"] ∈ Set(["trunk", "primary"]) && haskey(way.tags, "ref"))
    )

# don't remove islands as this will remove all freeways that go to the border past the last point where you can
# turn around without using the motorway system, as these are in weak components but not strong components.
G = OpenStreetMapGraphBuilder.OSM.build_graph("../data/california-latest.osm.pbf",
    way_filter=way_filter, turn_restrictions=false, remove_islands=0)

┌ Info: Pass 1: find intersection nodes
└ @ OpenStreetMapGraphBuilder.OSM /Users/mwbc/.julia/packages/OpenStreetMapGraphBuilder/mhpEx/src/osm/build_graph.jl:26
┌ Info: Reading file written by osmium/1.8.0
└ @ OpenStreetMapPBF /Users/mwbc/.julia/packages/OpenStreetMapPBF/kl2S6/src/scan.jl:202
┌ Info: ..parsed 102965 ways
└ @ OpenStreetMapGraphBuilder.OSM /Users/mwbc/.julia/packages/OpenStreetMapGraphBuilder/mhpEx/src/osm/build_graph.jl:37
┌ Info: ..found 90773 intersection nodes
└ @ OpenStreetMapGraphBuilder.OSM /Users/mwbc/.julia/packages/OpenStreetMapGraphBuilder/mhpEx/src/osm/build_graph.jl:49
┌ Info: Pass 2: read intersection and other highway nodes
└ @ OpenStreetMapGraphBuilder.OSM /Users/mwbc/.julia/packages/OpenStreetMapGraphBuilder/mhpEx/src/osm/build_graph.jl:51
┌ Info: Reading file written by osmium/1.8.0
└ @ OpenStreetMapPBF /Users/mwbc/.julia/packages/OpenStreetMapPBF/kl2S6/src/scan.jl:202
┌ Info: Pass 3: re-read and catalog ways
└ @ OpenStreetMapGraphBuilder.OSM /Users/mwbc

{128584, 141375} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [80]:
# serialize so can be viewed with StreetRouterVisualizer
serialize("../data/ca_highways.gr", G)

## Read sensors

In [81]:
sensors = CSV.read("../data/stations_used_in_analysis.csv", DataFrame);

## Build a spatial index of the vertices

Edge-based graph - vertices are links.

In [82]:
rtree = LibSpatialIndex.RTree(2)

LibSpatialIndex.RTree(Ptr{Nothing} @0x0000600002214b80, LibSpatialIndex.C.RT_RTree, 0x00000002, LibSpatialIndex.C.RT_Star, LibSpatialIndex.C.RT_Memory, 0x00000064, 0x00000064, 0x00000064, 0x00000064, 0x000003e8, 0x000001f4, true, 0x00000020, 0.7, 0.4, 0.3)

In [83]:
bbox(geom) = [minimum(map(x -> x.lon, geom)), minimum(map(x -> x.lat, geom))],
    [maximum(map(x -> x.lon, geom)), maximum(map(x -> x.lat, geom))]

bbox (generic function with 1 method)

In [84]:
for v in vertices(G)
    LibSpatialIndex.insert!(rtree, v, bbox(get_prop(G, v, :geom))...)
end

## Snap each sensor to the nearest highway

In [98]:
# LibSpatialIndex query not thread safe, fwiw
sensors.vertex = map(zip(sensors.station, sensors.Latitude, sensors.Longitude)) do (station, lat, lon)
    coslat = cosd(lat)
    search_radius_degrees_lon = SEARCH_RADIUS_DEGREES_LAT / cosd(lat)
    candidates = LibSpatialIndex.intersects(
        rtree,
        [lon - search_radius_degrees_lon, lat - SEARCH_RADIUS_DEGREES_LAT],
        [lon + search_radius_degrees_lon, lat + SEARCH_RADIUS_DEGREES_LAT]
    )
    
    best_vertex::Union{Int64, Missing} = missing
    
    # results are in lat degrees
    # Clamp to radius to avoid grabbing something that was in a corner when there
    # might be something closer not in the corner.
    best_distance::Float64 = SEARCH_RADIUS_DEGREES_LAT
    
    # is the best distance a motorway
    found_motorway = false
    
    sensor_geos = Point(lon * coslat, lat)
    
    # this part could potentially be multithreaded
    for candidate in candidates
        candidate_geom = get_prop(G, candidate, :geom)
        # simple spherical projection
        candidate_geos = LineString(map(x -> [x.lon * coslat, x.lat], candidate_geom))
        
        d = LibGEOS.distance(sensor_geos, candidate_geos)
        
        if d < SEARCH_RADIUS_DEGREES_LAT
            # this is a possible match
            # TODO should have class attached to vertex
            outnbrs = outneighbors(G, candidate)
            class = if !isempty(outnbrs)
                get_prop(G, candidate, first(outnbrs), :this_class)
            else
                innbrs = inneighbors(G, candidate)
                if !isempty(innbrs)
                    get_prop(G, first(innbrs), candidate, :next_class)
                else
                    continue # No in or out neighbors, don't snap to this
                end
            end
                
            if class == OpenStreetMapGraphBuilder.OSM.motorway
                if !found_motorway
                    found_motorway = true
                    best_distance = d
                    best_vertex = candidate
                else
                    if d < best_distance
                        best_distance = d
                        best_vertex = candidate
                    end
                end
            elseif !found_motorway
                # only consider non-motorways if no motorways found
                if d < best_distance
                    best_distance = d
                    best_vertex = candidate
                end
            end
        end
    end
    
    best_vertex
end

mean(ismissing.(sensors.vertex))

0.0

## Compute graph centrality metrics

In [99]:
# first, weight graph
OpenStreetMapGraphBuilder.compute_freeflow_weights!(G)

In [149]:
meta = CSV.read("../data/sensor_meta_geo.csv", DataFrame)
leftjoin!(sensors, select(meta, Not([:Latitude, :Longitude])), on=:station=>:ID)

,station,Latitude,Longitude,vertex,betweenness_centrality,network_link
,Int64,Float64,Float64,Int64,Float64,String
1,312134,38.4124,-121.484,120755,0.0468653,"LINESTRING (-121.484289 38.412421, -121.48272300000001 38.40475)"
2,312346,38.6343,-121.5,119785,0.00590868,"LINESTRING (-121.499686 38.63434, -121.494752 38.636913)"
3,312386,38.477,-121.426,38795,0.0114678,"LINESTRING (-121.425542 38.477038, -121.424321 38.475840000000005)"
4,312388,38.4815,-121.43,29933,0.0114811,"LINESTRING (-121.430345 38.481528, -121.43046500000001 38.481579)"
5,312420,38.4846,-121.434,29934,0.0112471,"LINESTRING (-121.433775 38.484562, -121.434425 38.485081)"
6,312422,38.4949,-121.445,120138,0.0114741,"LINESTRING (-121.444522 38.494881, -121.443882 38.494236)"
7,312513,38.5027,-121.453,29991,0.0114871,"LINESTRING (-121.45279 38.502687, -121.45300300000001 38.502866000000004)"
8,312514,38.5091,-121.459,120113,0.0114801,"LINESTRING (-121.459013 38.509079, -121.45871700000001 38.508559000000005)"
9,312520,38.5117,-121.46,121479,0.0114934,"LINESTRING (-121.46048 38.511704, -121.45983670000001 38.5105106)"


In [159]:
x = ones(1000, 3)

1000×3 Matrix{Float64}:
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 ⋮         
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0

In [171]:
# compute betweenness centrality by district
function group_centrality(vertices)
    # calc betweenness centrality - calcs for all vertices
    # not using Graphs.jl function as it always calculates for all destinations
    
    centralities = zeros(Int64, nv(G), Threads.nthreads())
    
    Threads.@threads for source in vertices
        ds = dijkstra_shortest_paths(G, source)
        
        for tgt in vertices
            if tgt != source && isfinite(ds.dists[tgt])
                current = ds.parents[tgt]
                
                while current != source
                    centralities[current, Threads.threadid()] += 1
                    current = ds.parents[current]
                end
            end
        end
    end
    
    (dropdims(sum(centralities, dims=2), dims=2) ./ (length(vertices) * (length(vertices) - 1)))[vertices]
end

@pipe groupby(sensors, :District) |>
    transform!(_, :vertex => group_centrality => :district_betweenness_centrality)

,station,Latitude,Longitude,vertex,betweenness_centrality,network_link
,Int64,Float64,Float64,Int64,Float64,String
1,312134,38.4124,-121.484,120755,0.0468653,"LINESTRING (-121.484289 38.412421, -121.48272300000001 38.40475)"
2,312346,38.6343,-121.5,119785,0.00590868,"LINESTRING (-121.499686 38.63434, -121.494752 38.636913)"
3,312386,38.477,-121.426,38795,0.0114678,"LINESTRING (-121.425542 38.477038, -121.424321 38.475840000000005)"
4,312388,38.4815,-121.43,29933,0.0114811,"LINESTRING (-121.430345 38.481528, -121.43046500000001 38.481579)"
5,312420,38.4846,-121.434,29934,0.0112471,"LINESTRING (-121.433775 38.484562, -121.434425 38.485081)"
6,312422,38.4949,-121.445,120138,0.0114741,"LINESTRING (-121.444522 38.494881, -121.443882 38.494236)"
7,312513,38.5027,-121.453,29991,0.0114871,"LINESTRING (-121.45279 38.502687, -121.45300300000001 38.502866000000004)"
8,312514,38.5091,-121.459,120113,0.0114801,"LINESTRING (-121.459013 38.509079, -121.45871700000001 38.508559000000005)"
9,312520,38.5117,-121.46,121479,0.0114934,"LINESTRING (-121.46048 38.511704, -121.45983670000001 38.5105106)"


In [173]:
CSV.write("../data/sensor_centrality.csv", sensors)

"../data/sensor_centrality.csv"

## Save links to network

So we can map how everything was connected.

In [111]:
sensors.network_link = map(zip(sensors.Latitude, sensors.Longitude, sensors.vertex)) do (lat, lon, vertex)
    v_geom = get_prop(G, vertex, :geom)
    v_centerish = v_geom[length(v_geom) ÷ 2]
    
    link_geom = LineString([[lon, lat], [v_centerish.lon, v_centerish.lat]])
    
    writegeom(link_geom)
end

3514-element Vector{String}:
 "LINESTRING (-121.484289 38.412421, -121.48272300000001 38.40475)"
 "LINESTRING (-121.499686 38.63434, -121.494752 38.636913)"
 "LINESTRING (-121.425542 38.477038, -121.424321 38.475840000000005)"
 "LINESTRING (-121.430345 38.481528, -121.43046500000001 38.481579)"
 "LINESTRING (-121.433775 38.484562, -121.434425 38.485081)"
 "LINESTRING (-121.444522 38.494881, -121.443882 38.494236)"
 "LINESTRING (-121.45279 38.502687, -121.45300300000001 38.502866000000004)"
 "LINESTRING (-121.459013 38.509079, -121.45871700000001 38.508559000000005)"
 "LINESTRING (-121.46048 38.511704, -121.45983670000001 38.5105106)"
 "LINESTRING (-121.473985 38.545423, -121.47388500000001 38.538628)"
 "LINESTRING (-121.405805 38.446758, -121.40759100000001 38.450536)"
 "LINESTRING (-121.409707 38.455428, -121.407966 38.451792000000005)"
 "LINESTRING (-121.46446 38.579355, -121.46375900000001 38.580865)"
 ⋮
 "LINESTRING (-117.86475 33.655401, -117.8631457 33.6539475)"
 "LINESTRING (-11

In [113]:
props(G, 1)

Dict{Symbol, Any} with 7 entries:
  :way           => 2022
  :heading_end   => 51.4749
  :nodes         => [1833121478, 278607482]
  :geom          => Geodesy.LLA{Float64}[LLA(lat=35.0683591°, lon=-116.4159934°…
  :from_node     => 1833121478
  :heading_start => 51.4749
  :to_node       => 278607482

In [112]:
CSV.write("../data/network_links.csv", select(sensors, [:station, :vertex, :network_link]))

"../data/network_links.csv"

In [119]:
# write the network to a shapefile
# https://discourse.julialang.org/t/how-to-create-a-new-shapefile-containing-a-few-points/43454/3
ArchGDAL.create("../data/network.gpkg", driver = ArchGDAL.getdriver("GPKG")) do ds
    # EPSG 4326 - WGS 84 - coordinate reference system used by OpenStreetMap
    ArchGDAL.createlayer(name="network", geom=ArchGDAL.wkbLineString, dataset=ds, spatialref=ArchGDAL.importEPSG(4326)) do layer
        ArchGDAL.addfielddefn!(layer, "vertexid", ArchGDAL.OFTInteger)
        
        for v in vertices(G)
            geom = get_prop(G, v, :geom)
            lats = [x.lat for x in geom]
            lons = [x.lon for x in geom]
            ArchGDAL.addfeature(layer) do f
                ArchGDAL.setgeom!(f, ArchGDAL.createlinestring(lons, lats))
                ArchGDAL.setfield!(f, 0, v)
            end
        end
    end
end

In [117]:
ArchGDAL.getdriver("GPKG")

Driver: GPKG/GeoPackage

## Build the weights matrix

In [129]:
# Create the weights matrix
# don't use a sparse matrix, not thread safe
wmat = zeros(Bool, nrow(sensors), nrow(sensors))

3514×3514 Matrix{Bool}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  

In [130]:
# now, find nearest four neighbors
Threads.@threads for (vidx, vertex) in collect(enumerate(sensors.vertex))
    ds = dijkstra_shortest_paths(G, vertex)
    # now compute dists to all sensors
    all_sensor_dists = map(x -> ds.dists[x], sensors.vertex)
    
    # sort it
    sorter = sortperm(all_sensor_dists)
    
    for tidx in 1:4
        if isfinite(all_sensor_dists[sorter[tidx]])
            wmat[vidx, sorter[tidx]] = true
        end
    end     
end

In [131]:
W = SpatialWeights(wmat)

Spatial Weights 
Observations: 3514 
Transformation: row
Minimum nunmber of neighbors: 1
Maximum nunmber of neighbors: 4
Average number of neighbors: 3.9903
Median number of neighbors: 4.0
Islands (isloated): 0
Density: 0.1136% 


In [132]:
changes = CSV.read("../data/stations_with_changes.csv", DataFrame)

,station,mean_flow_prepandemic,urban_prepandemic,district_prepandemic,lanes_prepandemic
,Int64,Float64,Bool,Int64,Int64
1,312133,42421.5,1,3,3
2,312134,33771.6,1,3,2
3,312346,63754.6,1,3,3
4,312386,64980.0,1,3,3
5,312388,71449.4,1,3,3
6,312420,78515.2,1,3,3
7,312422,64266.2,1,3,3
8,312513,82873.1,1,3,3
9,312514,72889.7,1,3,3


In [134]:
leftjoin!(sensors, select(changes, [:station, :peak_hour_occ_prepandemic, :peak_hour_occ_postlockdown]), on=:station)

,station,Latitude,Longitude,vertex,betweenness_centrality,network_link
,Int64,Float64,Float64,Int64,Float64,String
1,312134,38.4124,-121.484,120755,0.0468653,"LINESTRING (-121.484289 38.412421, -121.48272300000001 38.40475)"
2,312346,38.6343,-121.5,119785,0.00590868,"LINESTRING (-121.499686 38.63434, -121.494752 38.636913)"
3,312386,38.477,-121.426,38795,0.0114678,"LINESTRING (-121.425542 38.477038, -121.424321 38.475840000000005)"
4,312388,38.4815,-121.43,29933,0.0114811,"LINESTRING (-121.430345 38.481528, -121.43046500000001 38.481579)"
5,312420,38.4846,-121.434,29934,0.0112471,"LINESTRING (-121.433775 38.484562, -121.434425 38.485081)"
6,312422,38.4949,-121.445,120138,0.0114741,"LINESTRING (-121.444522 38.494881, -121.443882 38.494236)"
7,312513,38.5027,-121.453,29991,0.0114871,"LINESTRING (-121.45279 38.502687, -121.45300300000001 38.502866000000004)"
8,312514,38.5091,-121.459,120113,0.0114801,"LINESTRING (-121.459013 38.509079, -121.45871700000001 38.508559000000005)"
9,312520,38.5117,-121.46,121479,0.0114934,"LINESTRING (-121.46048 38.511704, -121.45983670000001 38.5105106)"


In [136]:
sensors.Δpeak_hour_occ = sensors.peak_hour_occ_postlockdown .- sensors.peak_hour_occ_prepandemic;

In [137]:
moran(sensors.Δpeak_hour_occ, W, permutations=9999)

Moran's I test of Global Spatial Autocorrelation
--------------------------------------------

Moran's I: 0.5445528
Expectation: -0.0002847

Randomization test with 9999 permutations.
 Mean: 0.2503176
 Std Error: 0.0078943
 zscore: 37.2719107
 p-value: 0.0001
